In [1]:
from transformers import Trainer, BertTokenizer, BertForSequenceClassification
from datasets import load_from_disk
import optuna
import torch
import math
import base

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 1g.10gb


In [3]:
DATASET = "sst2"

In [4]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented")
tokenizer = BertTokenizer.from_pretrained("ndavid/autotrain-trec-fine-bert-739422530")

In [5]:
train = train_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the train dataset")
eval = eval_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the eval dataset")
test = test_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the test dataset")

train_aug = all_train_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the augmented dataset")

In [6]:
num_epochs = 15
batch_size = 128

In [7]:
#Nápočet epoch na steps
data_length = len(train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [8]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

In [9]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [10]:
def get_Bert():
    return BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=2)

In [11]:
base.reset_seed()

In [12]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base_hp-search", logging_dir=f"~/logs/{DATASET}/bert-base_hp-search", epochs=num_epochs, batch_size=batch_size)

In [13]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert(),
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-base",
    n_trials=150
)

[I 2025-03-28 11:31:45,321] A new study created in memory with name: Test-base


Trial 0 with params: {'learning_rate': 4.3284502212938785e-05, 'weight_decay': 0.01, 'warmup_steps': 32}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.518900,0.461005,0.793578,0.793637,0.793340,0.793421
2,0.362800,0.434238,0.809633,0.809585,0.809527,0.809552
3,0.305700,0.436370,0.815367,0.815301,0.815326,0.815312
4,0.272100,0.454227,0.822248,0.822380,0.821998,0.822101
5,0.246200,0.441469,0.810780,0.810802,0.810906,0.810768
6,0.227400,0.461721,0.814220,0.814494,0.814494,0.814220
7,0.214000,0.481988,0.813073,0.813301,0.813326,0.813073
8,0.202600,0.489547,0.814220,0.814175,0.814116,0.814141
9,0.192200,0.500466,0.808486,0.809646,0.809032,0.808444
10,0.185200,0.505199,0.816514,0.816456,0.816536,0.816479


[I 2025-03-28 11:38:30,533] Trial 0 pruned. 


...

Trial 137 with params: {'learning_rate': 3.225367657608565e-05, 'weight_decay': 0.007, 'warmup_steps': 43}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.548600,0.469761,0.788991,0.788957,0.788836,0.788880
2,0.391100,0.442409,0.801606,0.801600,0.801433,0.801490
3,0.335000,0.434901,0.810780,0.810926,0.810990,0.810778
4,0.302200,0.446904,0.815367,0.815762,0.814989,0.815133


[W 2025-03-28 23:18:37,897] Trial 137 failed with parameters: {'learning_rate': 3.225367657608565e-05, 'weight_decay': 0.007, 'warmup_steps': 43} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations/integration_utils.py", line 250, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2241, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2548, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs, num_items_in_batch)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 3698, in training_step
    loss = self.compute_loss(model,

KeyboardInterrupt: 

In [15]:
print(best_trial)

NameError: name 'best_trial' is not defined

In [34]:
base.reset_seed()

In [35]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill_hp-search", logging_dir=f"~/logs/{DATASET}/bert-distill_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [36]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [37]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [38]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert(),
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
best_trial2 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distilation",
    n_trials=150
)

[I 2025-03-29 00:46:31,583] A new study created in memory with name: Distilation


Trial 0 with params: {'learning_rate': 4.3284502212938785e-05, 'weight_decay': 0.01, 'warmup_steps': 32, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.642600,1.794711,0.795872,0.795997,0.795592,0.795690
2,1.683600,1.584951,0.795872,0.796962,0.795256,0.795397
3,1.307100,1.489271,0.809633,0.809575,0.809653,0.809597
4,1.098600,1.506980,0.811927,0.812163,0.811611,0.811732
5,0.957400,1.476046,0.817661,0.817612,0.817704,0.817632
6,0.867000,1.505150,0.815367,0.815445,0.815536,0.815361
7,0.798300,1.563696,0.814220,0.814494,0.814494,0.814220
8,0.748800,1.550323,0.814220,0.814206,0.814073,0.814122
9,0.705200,1.591043,0.819954,0.820280,0.820251,0.819954
10,0.673100,1.564754,0.821101,0.821035,0.821083,0.821055


[I 2025-03-29 00:50:40,600] Trial 0 pruned. 


...

[I 2025-03-29 12:32:16,844] Trial 148 finished with value: 0.8221801222215643 and parameters: {'learning_rate': 6.974273840162806e-05, 'weight_decay': 0.004, 'warmup_steps': 16, 'lambda_param': 0.1, 'temperature': 2.0}. Best is trial 37 with value: 0.8267526114341277.


Trial 149 with params: {'learning_rate': 8.221432133065498e-05, 'weight_decay': 0.006, 'warmup_steps': 1, 'lambda_param': 0.2, 'temperature': 7.0}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.240000,1.609083,0.813073,0.813025,0.813116,0.813044
2,1.280200,1.484442,0.810780,0.810723,0.810695,0.810708
3,0.951600,1.500191,0.818807,0.818856,0.818957,0.818799
4,0.789700,1.602838,0.825688,0.828375,0.824830,0.825017
5,0.679100,1.622210,0.816514,0.816789,0.816789,0.816514
6,0.611900,1.617079,0.817661,0.818217,0.818041,0.817655
7,0.549600,1.679772,0.810780,0.810802,0.810906,0.810768
8,0.511000,1.683468,0.817661,0.817721,0.817452,0.817534
9,0.471100,1.706438,0.815367,0.816732,0.815957,0.815312
10,0.447500,1.734621,0.816514,0.816456,0.816536,0.816479


[I 2025-03-29 12:38:29,633] Trial 149 finished with value: 0.8210773868712218 and parameters: {'learning_rate': 8.221432133065498e-05, 'weight_decay': 0.006, 'warmup_steps': 1, 'lambda_param': 0.2, 'temperature': 7.0}. Best is trial 37 with value: 0.8267526114341277.


In [40]:
print(best_trial2)

BestRun(run_id='37', objective=0.8267526114341277, hyperparameters={'learning_rate': 4.729948829550423e-05, 'weight_decay': 0.002, 'warmup_steps': 9, 'lambda_param': 0.8, 'temperature': 2.0}, run_summary=None)


In [13]:
#Nápočet epoch na steps
data_length = len(train_aug)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [14]:
base.reset_seed()

In [15]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bert-base_aug_hp-search", epochs=num_epochs, batch_size=batch_size)

In [16]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [17]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [18]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert()
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
best_trial3 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-base-aug",
    n_trials=150
)

[I 2025-03-29 17:37:29,505] A new study created in memory with name: Test-base-aug


Trial 0 with params: {'learning_rate': 4.3284502212938785e-05, 'weight_decay': 0.01, 'warmup_steps': 169}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.343500,0.427888,0.810780,0.812774,0.811495,0.810670
2,0.202500,0.470815,0.818807,0.818765,0.818704,0.818730
3,0.164500,0.521956,0.802752,0.802823,0.802517,0.802603
4,0.141800,0.555934,0.806193,0.806338,0.806401,0.806190
5,0.125800,0.617021,0.799312,0.799351,0.799097,0.799172


[I 2025-03-29 17:56:03,729] Trial 0 pruned. 


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.454100,0.433555,0.803899,0.804878,0.804401,0.803868
2,0.311000,0.419960,0.810780,0.811945,0.811327,0.810738
3,0.262100,0.419574,0.818807,0.819302,0.819167,0.818804
4,0.232400,0.433600,0.825688,0.825737,0.825840,0.825680
5,0.212500,0.441854,0.821101,0.821066,0.821167,0.821077
6,0.198800,0.456657,0.823394,0.823338,0.823419,0.823361
7,0.187300,0.465700,0.818807,0.818856,0.818957,0.818799
8,0.179400,0.482230,0.816514,0.816501,0.816368,0.816417
9,0.173100,0.492106,0.817661,0.817631,0.817536,0.817574
10,0.168900,0.496024,0.817661,0.817683,0.817789,0.817649


[I 2025-03-30 22:58:15,340] Trial 41 finished with value: 0.8107497010902807 and parameters: {'learning_rate': 1.2775087742305899e-05, 'weight_decay': 0.001, 'warmup_steps': 157}. Best is trial 23 with value: 0.8164664530353019.


Trial 42 with params: {'learning_rate': 1.0890360023105325e-05, 'weight_decay': 0.002, 'warmup_steps': 119}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.468000,0.437943,0.800459,0.801351,0.800939,0.800432
2,0.326500,0.420551,0.813073,0.814432,0.813663,0.813018
3,0.277700,0.418631,0.816514,0.816891,0.816831,0.816513
4,0.247400,0.428055,0.823394,0.823506,0.823588,0.823391
5,0.226500,0.432392,0.821101,0.821101,0.821209,0.821086
6,0.212300,0.441739,0.826835,0.826816,0.826924,0.826816
7,0.200100,0.450301,0.822248,0.822326,0.822419,0.822242
8,0.192000,0.463090,0.825688,0.825629,0.825629,0.825629


In [ ]:
print(best_trial3)

NameError: name 'best_trial3' is not defined

In [ ]:
base.reset_seed()

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bert-distill_aug_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [ ]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [ ]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert()
)
  

NameError: name 'BertForSequenceClassification' is not defined

In [ ]:
best_trial4 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-Distill-aug",
    n_trials=150
)

In [ ]:
print(best_trial4)

NameError: name 'best_trial4' is not defined

In [ ]:
print("Best normal training score: ", best_trial)
print("Best distilation trianing score: ", best_trial2)
#print("Best normal training score with augmentations: ", best_trial3)
print("Best distilation trianing score with augmentations: ",best_trial4)

Best normal training score:  BestRun(run_id='132', objective=0.49578714001038604, hyperparameters={'learning_rate': 0.0004675471848767979, 'weight_decay': 0.01, 'warmup_steps': 4}, run_summary=None)
Best distilation trianing score:  BestRun(run_id='86', objective=0.4778879458794155, hyperparameters={'learning_rate': 0.00048481023093695626, 'weight_decay': 0.003, 'warmup_steps': 4, 'lambda_param': 0.4, 'temperature': 2.5}, run_summary=None)
Best distilation trianing score with augmentations:  BestRun(run_id='92', objective=0.7644517643387146, hyperparameters={'learning_rate': 0.0004922578519032032, 'weight_decay': 0.008, 'warmup_steps': 6, 'lambda_param': 1.0, 'temperature': 4.0}, run_summary=None)
